In [1]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from catboost import Pool, cv
random_state = 7
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import os
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import warnings
import numpy as np
warnings.filterwarnings("ignore")

In [2]:
def model_validation(tr,ts, iterations, learning_rate,depth, n_splits = 10):
   
    # get current directory
    path = os.getcwd()
    parent = os.path.dirname(path)

    train = pd.read_csv(parent + '\Data\\' + tr + ".csv")
    test = pd.read_csv(parent + '\Data\\' + ts + ".csv")
    
    X_train = train.drop("target", axis=1)
    y_train = train["target"]
    
    X_test  = test.drop("target", axis=1)
    y_test = test["target"]
    
        # initialize Pool
    train_pool = Pool(X_train, 
                      y_train)
        # initialize Pool
    test_pool = Pool(X_test, 
                      y_test)
    
    scores = pd.DataFrame(columns=["Iterations", "Learning Rate", "Depth", "F1", "ROC", "Sensitivity", "Specificty", "CV F1", "CV ROC", "ROC Sens", "ROC Spec"])
    
#     from sklearn.linear_model import LogisticRegression
#     clf = LogisticRegression(random_state=0, penalty="l1", solver='liblinear')
#     clf.fit(X_train, y_train)
#     pred = clf.predict(X_test)
#     f1 = (f1_score(pred, y_test))

#     cm = confusion_matrix(y_test, pred)
#     total=sum(sum(cm))
#     roca = round(roc_auc_score(y_test, pred),4)
#     sens = round(cm[1,1]/(cm[1,1]+cm[0,1]),4)
#     spec = round(cm[0,0]/(cm[1,0]+cm[0,0]),4)      
    
#     print(f1, roca, sens, spec)

    for iteration in iterations:
        for rate in learning_rate:
            for deep in depth:
                    start = time.time()
                    model = CatBoostClassifier(iterations=iteration,
                           depth=deep,
                           learning_rate=rate,
                           loss_function='Logloss',
                           verbose=False)
                    
                    model.fit(train_pool)
                    pred = model.predict(X_test)
                    f1 = (f1_score(pred, y_test))
                    
                    cm = confusion_matrix(y_test, pred)
                    total=sum(sum(cm))
                    roca = round(roc_auc_score(y_test, pred),4)
                    sens = round(cm[1,1]/(cm[1,1]+cm[0,1]),4)
                    spec = round(cm[0,0]/(cm[1,0]+cm[0,0]),4)                    
                    
                    
                    
                    scores_cv = []
                    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
                    skf.get_n_splits(X_train, y_train)
                    for train_index, test_index in skf.split(X_train,y_train):
                        X_cv_train, X_cv_test = X_train.iloc[train_index], X_train.iloc[test_index]
                        y_cv_train, y_cv_test = y_train.iloc[train_index], y_train.iloc[test_index]

                        model_cv = CatBoostClassifier(iterations=iteration,
                                               depth=deep,
                                               learning_rate=rate,
                                               loss_function='Logloss',
                                               verbose=False)

                        # initialize Pool
                        train_pool = Pool(X_cv_train, 
                                          y_cv_train)
                            # initialize Pool
                        test_pool = Pool(X_cv_test, 
                                          y_cv_test)

                        model_cv.fit(train_pool)
                        pred = model_cv.predict(X_cv_test)
                        f1_cv = (f1_score(pred, y_cv_test))

                        cm = confusion_matrix(y_cv_test, pred)
                        total=sum(sum(cm))
                        roca_cv = round(roc_auc_score(y_cv_test, pred),4)
                        sens_cv = round(cm[1,1]/(cm[1,1]+cm[0,1]),4)
                        spec_cv = round(cm[0,0]/(cm[1,0]+cm[0,0]),4) 
                        
                        scores_cv.append([f1_cv, roca_cv, sens_cv, spec_cv])

                      
                    
                    mean = (np.mean(np.array(scores_cv), axis=0))
                    df_new_row = pd.DataFrame(data=([[iteration, rate, deep, f1, roca, sens, spec, mean[0], mean[1], mean[2], mean[3]]]), 
                                              columns=["Iterations", "Learning Rate", "Depth",  "F1", "ROC", "Sensitivity", "Specificty",
                                                      "CV F1", "CV ROC", "ROC Sens", "ROC Spec"])
                    scores = pd.concat([scores,df_new_row], ignore_index=True)
                    end = time.time()
                    print(end - start)
                    
            
    return scores

In [29]:
tr = "testset_ALL_vs_Healthy_reduced_75_20_target_in"
ts = "trainset_ALL_vs_Healthy_reduced_75_80_target_in"

# tr = "trainset_AML_vs_ALL_reduced_90_target_in"
# ts = "testset_AML_vs_ALL_reduced_90_target_in"

start = time.time()

#  iterations, learning_rate,depth, weights, n_splits = 10
results = model_validation(tr,ts, [100 ,200 ,300 ,400], [0.3, 0.5, 0.7, 1],[3, 4, 5, 6, 7])




1.4817681312561035
1.9307284355163574
2.6655373573303223
4.510477781295776
8.205089807510376
1.5193383693695068
2.210197687149048
3.105485200881958
4.993347644805908
8.687868356704712
1.6777369976043701
2.094583034515381
3.0452418327331543
4.932566165924072
9.079526662826538
1.5031461715698242
1.983208179473877
2.930534601211548
4.8248772621154785
10.558537483215332
3.4026780128479004
5.150944232940674
7.089087963104248
11.813552141189575
20.56347346305847
3.4069983959198
4.901175260543823
6.9113383293151855
11.627806663513184
20.24312710762024
3.336228370666504
4.892328977584839
7.101542711257935
11.591951131820679
20.405476093292236
3.360106945037842
5.135555267333984
7.049859285354614
11.68327784538269
20.466707944869995
5.0652289390563965
7.3477842807769775
10.346123456954956
17.877007484436035
32.0583279132843
5.3701324462890625
7.100610733032227
11.290860652923584
18.604037046432495
31.507609844207764
4.974505186080933
7.079352855682373
10.94932508468628
17.07998514175415
30.0828

In [30]:
index = 0
top = 0
i = 0
for row in results.loc[:]["CV F1"]:
    if row > top:
        top = row
        index = i
    i = i + 1
print(results.loc[index])
for item in results.loc[index]:
    print(item)

Iterations            100
Learning Rate         0.3
Depth                   4
F1               0.994946
ROC                0.9888
Sensitivity        0.9957
Specificty         0.9781
CV F1            0.994964
CV ROC             0.9891
ROC Sens          0.99575
ROC Spec          0.97866
Name: 1, dtype: object
100
0.3
4
0.9949458483754512
0.9888
0.9957
0.9781
0.994963578667827
0.9891
0.9957499999999999
0.97866
